# 5 - Cadena con Flask

<br>
<br>

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/flask_api.webp" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---Cadena-de-LangChain" data-toc-modified-id="1---Cadena-de-LangChain-1">1 - Cadena de LangChain</a></span></li><li><span><a href="#2---Cadena-en-una-función" data-toc-modified-id="2---Cadena-en-una-función-2">2 - Cadena en una función</a></span></li><li><span><a href="#3---Código-de-Flask" data-toc-modified-id="3---Código-de-Flask-3">3 - Código de Flask</a></span></li></ul></div>

## 1 - Cadena de LangChain

Vamos a usar una cadena de Langchain sencilla para usarla dentro del ejemplo anterior. Usaremos una cadena que ya hemos visto anteriormente para usarla dentro la aplicación de Flask. Vamos a ver otra vez esa cadena paso a paso y crear una función con ella.

In [ ]:
# primero cargamos la API KEY de OpenAI

from dotenv import load_dotenv 
import os

# carga de variables de entorno
load_dotenv()


# api key openai, nombre que tiene por defecto en LangChain
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
# preparamos el prompt

from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages([
    
    ('system', '''Eres un historiador muy erudito que ofrece respuestas precisas y 
                  elocuentes a preguntas históricas y que responde en castellano.'''),
    
    ('human', '{pregunta}')
    
])

In [ ]:
# iniciamos el modelo llm

from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

In [ ]:
# parser de salida, transforma la salida a string

from langchain.schema import StrOutputParser

parser = StrOutputParser()

In [ ]:
# creamos la cadena con lcel

cadena = prompt | modelo | parser

In [ ]:
# llamada a la cadena

pregunta = '¿Cuales son las 7 maravillas del mundo?'

respuesta = cadena.invoke({'pregunta': pregunta})

In [ ]:
respuesta

## 2 - Cadena en una función

Ahora que hemos visto esa cadena sencilla paso a paso, vamos a crear una función que reciba la pregunta del usuario y devuelva la respuesta del modelo. Esta función la usaremos después en la aplicación de Flask que creamos anteriormente.

In [ ]:
# importamos librerías

from dotenv import load_dotenv 
import os

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser



# carga de variables de entorno
load_dotenv()

# api key openai, nombre que tiene por defecto en LangChain
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')



def cadena_llm(pregunta:str) -> str:
    
    """
    Esta función es para el uso de un LLM con una cadena sencilla:
    
    Params:
    pregunta: str, inoput del usuario
    
    Return:
    string, devuelve la pregunta del modelo
    """
    
    global OPENAI_API_KEY
    
    # preparamos el prompt
    prompt = ChatPromptTemplate.from_messages([

        ('system', '''Eres un historiador muy erudito que ofrece respuestas precisas y 
                      elocuentes a preguntas históricas y que responde en castellano.'''),

        ('human', '{pregunta}')

    ])


    # iniciamos el modelo llm
    modelo = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

    # parser de salida, transforma la salida a string
    parser = StrOutputParser()
    
    # creamos la cadena con lcel
    cadena = prompt | modelo | parser
    
    # llamada a la cadena
    respuesta = cadena.invoke({'pregunta': pregunta})
    
    return respuesta

In [ ]:
cadena_llm(pregunta)

## 3 - Código de Flask

Ahora vamos a usar esta función para la página de chat que creamos anteriormente. En cuanto la aplicación realice la petición POST y reciba el mensaje del usuario, dicho mensaje se usará para crear la respuesta del modelo y tanto mensaje como respuesta se devolverán al HTML. El objetivo es crear un chat 

In [ ]:
# importamos librerias
from flask import Flask, render_template, request, redirect, url_for
from dotenv import load_dotenv 
import os

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser



# carga de variables de entorno
load_dotenv()

# api key openai, nombre que tiene por defecto en LangChain
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


# iniciamos la aplicacion
app = Flask(__name__)

# almacén temporal de mensajes, en memoria
mensajes = []


# funcion del modelo
def cadena_llm(pregunta:str) -> str:
    
    """
    Esta función es para el uso de un LLM con una cadena sencilla:
    
    Params:
    pregunta: str, inoput del usuario
    
    Return:
    string, devuelve la pregunta del modelo
    """
    
    global OPENAI_API_KEY
    
    # preparamos el prompt
    prompt = ChatPromptTemplate.from_messages([

        ('system', '''Eres un historiador muy erudito que ofrece respuestas precisas y 
                      elocuentes a preguntas históricas y que responde en castellano.'''),

        ('human', '{pregunta}')

    ])


    # iniciamos el modelo llm
    modelo = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

    # parser de salida, transforma la salida a string
    parser = StrOutputParser()
    
    # creamos la cadena con lcel
    cadena = prompt | modelo | parser
    
    # llamada a la cadena
    respuesta = cadena.invoke({'pregunta': pregunta})
    
    return respuesta



# ruta principal que muestra el chat
@app.route('/', methods=['GET', 'POST'])
def chat():
    
    global cadena_llm
    
    # si el metodo es POST..
    if request.method == 'POST':
        
        # se obtiene el mensaje
        mensaje = request.form.get('mensaje')
        
        # respuesta del LLM
        respuesta = cadena_llm(mensaje)
        
        # se añade a la lista
        mensajes.append({'nombre': 'Tu', 'mensaje': mensaje})
        mensajes.append({'nombre': 'Bot', 'mensaje': respuesta})
        
        # redirige para evitar reenviar el formulario
        return redirect(url_for('chat'))  
    
    # renderiza el html
    return render_template('chat.html', mensajes=mensajes)


if __name__ == '__main__':
    app.run(debug=False, port=5001)
